## PyTorch Implementation of the Transformers Architecture

We will implement the transformers architecture in this notebook. We will also have 2-3 other notebooks coverying other forms of attention as well some relevant details for our future implementation go GCViT. 

### Encoder Block: (N = 6)
1. Input Embedding (completed)
2. Positional Encoding (completed)
3. Multi-Head Attention (completed)
4. Add & Layer Normalization (completed)
5. Feed Forward (completed)
6. Skip Connections (completed)

In [2]:
# Importing the necessary modules. 
import math
import torch
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}") 

Using device: cuda


In [3]:
# Our input embeddings will have d_model (dimensionality) and our vocab_size as its variables. 

class InputEmbeddings(nn.Module): 
    def __init__(self, d_model: int, vocab_size: int): 
        super().__init__()
        self.d_model = d_model 
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x): 
        return self.embedding(x) * math.sqrt(self.d_model)

In [4]:
# This is for the model to have some information about the relative position of each word in a sentence.

class PositionalEncoding(nn.Module): 
    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        denom = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * denom)
        pe[:, 1::2] = torch.cos(position * denom)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x): 
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # Not a learnable parameter
        return self.dropout(x)

In [5]:
class LayerNormalization(nn.Module): 
    def __init__(self, eps: float = 10**-6) -> None: 
        super().__init__()
        self.alpha = nn.Parameter(torch.ones(1))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x): 
        mean = x.mean(dim = -1, keepdim=True)
        std = x.std(dim = -1, keepdim=True)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

In [6]:
class FeedForward(nn.Module): 
    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None: 
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) 

    
    def forward(self, x): 
        x = self.linear_1(x)
        x = self.dropout(torch.relu(x))
        x = self.linear_2(x)
        return x

In [7]:
class MHA(nn.Module): 
    def __init__(self, d_model: int, n_heads: int, dropout: float) -> None: 
        super().__init__()
        # Create Q, K, V matrix. 
        self.d_model = d_model
        self.n_heads = n_heads
        self.dropout = nn.Dropout(dropout)

        assert d_model % n_heads == 0, "d_model is not divisible by n_heads"

        self.d_k = d_model // n_heads
        
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
    
        self.w_o = nn.Linear(d_model, d_model)

    
    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout): 
        d_k = query.shape[-1]

        att_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None: 
            att_scores.masked_fill_(mask == 0, -1e10)
        att_scores = att_scores.softmax(dim = -1) # (batch, h, seq_len, seq_len)
        if dropout is not None: 
            att_scores = dropout(att_scores)
        return (att_scores @ value), att_scores

    
    def forward(self, q, k, v, mask): 
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        x, self.att_scores = MHA.attention(query, key, value, mask, self.dropout)

        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
        return self.w_o(x)

In [8]:
class SkipConnections(nn.Module): 
    def __init__(self, dropout: float) -> None: 
        super().__init__()
        self.dropout = nn.Dropout(dropout) 
        self.norm = LayerNormalization()

    def forward(self, x, sub): 
        y = self.norm(x)
        y = sublayer(y)
        y = self.dropout(y)
        return x + y

In [9]:
class EncoderBlock(nn.Module): 
    def __init__(self, self_att: MHA, feed_forw: FeedForward, dropout: float) -> None: 
        super().__init__()
        self.self_att = self_att
        self.feed_forw = feed_forw
        self.dropout = nn.Dropout(dropout)
        self.skip_conn = nn.ModuleList([SkipConnections(dropout) for _ in range(2)])
    def forward(sel, x, src_mask):
        x = self.skip_conn[0](x, lambda x: self.self_att(x, x, x, src_mask))
        x = self.skip_conn[1](x, self.feed_forw)
        return x

In [10]:
class Encoder(nn.Module): 
    def __init__(self, layers: nn.ModuleList) -> None: 
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, mask): 
        for layer in self.layers: 
            x = layer(x, mask)
        return self.norm(x)
        

## Decoder Block 

We have all the components that we need, now, we just have to combine the Decoder block and complete our architecture. 

In [18]:
class DecoderBlock(nn.Module): 
    def __init__(self, masked_att: MHA, cross_att: MHA, feed_forw: FeedForward, dropout: float) -> None: 
        super().__init__()
        self.masked_att = masked_att
        self.cross_att = cross_att
        self.feed_forw = feed_forw
        self.dropout = nn.Dropout(dropout)
        self.skip_conn = nn.ModuleList([SkipConnections(dropout) for _ in range(3)])

    def forward(self, x, enc_output, src_mask, tgt_mask): 
        x = self.skip_conn[0](x, lambda x: self.masked_att(x, x, x, tgt_mask))
        x = self.skip_conn[1](x, lambda x: self.cross_att(x, enc_output, enc_output, src_mask))
        x = self.skip_conn[2](x, self.feed_forw)
        return x

In [12]:
class Decoder(nn.Module): 
    def __init__(self, layers: nn.ModuleList) -> None: 
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, enc_output, src_mask, tgt_mask): 
        for layer in self.layers: 
            x = layer(x, enc_output, src_mask, tgt_mask)
        return self.norm(x)

In [13]:
class Projection(nn.Module): 
    def __init__(self, d_model: int, vocab_size: int) -> None: 
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x): 
        return torch.log_softmax(self.proj(x), dim=-1)

## Transformer

Now that we have everything ready, we will combine all these blocks and complete our architecture. 

In [14]:
class Transformer(nn.Module): 
    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, 
                 src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, proj: Projection): 
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_post = tgt_pos
        self.proj = proj

    def encode(self, src, src_mask): 
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decoder(self, enc_output, src_mask, tgt, tgt_mask): 
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, enc_output, src_mask, tgt_mask)

    def project(self, x): 
        return self.proj(x)

In [15]:
# We will connect all these blocks now. 

def build(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int = 512,
          layers: int = 6, heads: int = 8, dropout: float = 0.01, d_ff: int = 2048): 
    
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    enc_blocks = []
    for _ in range(layers): 
        self_att_block = MHA(d_model, heads, dropout)
        feed_forw_block = FeedForward(d_model, d_ff, dropout)
        enc_block = EncoderBlock(self_att_block, feed_forw_block, dropout)
        enc_blocks.append(enc_block)

    dec_blocks = []
    for _ in range(layers): 
        self_att_block = MHA(d_model, heads, dropout)
        cross_att_block = MHA(d_model, heads, dropout)
        feed_forw_block = FeedForward(d_model, d_ff, dropout)
        dec_block = DecoderBlock(self_att_block, cross_att_block, feed_forw_block, dropout)
        dec_blocks.append(dec_block)

    encoder = Encoder(nn.ModuleList(enc_blocks))
    decoder = Decoder(nn.ModuleList(dec_blocks))

    proj_layer = Projection(d_model, tgt_vocab_size)

    # Finally
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, proj_layer)

    # Xavier Initialization: 
    for p in transformer.parameters(): 
        if p.dim() > 1: 
            nn.init.xavier_uniform_(p)
    return transformer

In [19]:
transformer = build(512, 64, 6, 3)
print("Model's state_dict: ") 
for param_tensor in transformer.state_dict(): 
    print(param_tensor, "\t", transformer.state_dict()[param_tensor].size())

Model's state_dict: 
encoder.layers.0.self_att.w_q.weight 	 torch.Size([512, 512])
encoder.layers.0.self_att.w_q.bias 	 torch.Size([512])
encoder.layers.0.self_att.w_k.weight 	 torch.Size([512, 512])
encoder.layers.0.self_att.w_k.bias 	 torch.Size([512])
encoder.layers.0.self_att.w_v.weight 	 torch.Size([512, 512])
encoder.layers.0.self_att.w_v.bias 	 torch.Size([512])
encoder.layers.0.self_att.w_o.weight 	 torch.Size([512, 512])
encoder.layers.0.self_att.w_o.bias 	 torch.Size([512])
encoder.layers.0.feed_forw.linear_1.weight 	 torch.Size([2048, 512])
encoder.layers.0.feed_forw.linear_1.bias 	 torch.Size([2048])
encoder.layers.0.feed_forw.linear_2.weight 	 torch.Size([512, 2048])
encoder.layers.0.feed_forw.linear_2.bias 	 torch.Size([512])
encoder.layers.0.skip_conn.0.norm.alpha 	 torch.Size([1])
encoder.layers.0.skip_conn.0.norm.bias 	 torch.Size([1])
encoder.layers.0.skip_conn.1.norm.alpha 	 torch.Size([1])
encoder.layers.0.skip_conn.1.norm.bias 	 torch.Size([1])
encoder.layers.1.sel